Esteban Castañeda Blanco C01795
Israel López Vallecillo C04396
Daniel Lizano Morales C04285

In [3]:
import os
import shutil
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
base_dir = 'Plant_leave_diseases_dataset'
classes = [
  'Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Background_without_leaves',
  'Blueberry___healthy',
  'Cherry___healthy',
  'Cherry___Powdery_mildew',
  'Corn___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn___Common_rust',
  'Corn___healthy',
  'Corn___Northern_Leaf_Blight',
  'Grape___Black_rot',
  'Grape___Esca_(Black_Measles)',
  'Grape___healthy',
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
  'Orange___Haunglongbing_(Citrus_greening)',
  'Peach___Bacterial_spot',
  'Peach___healthy',
  'Pepper,_bell___Bacterial_spot',
  'Pepper,_bell___healthy',
  'Potato___Early_blight',
  'Potato___healthy',
  'Potato___Late_blight',
  'Raspberry___healthy',
  'Soybean___healthy',
  'Squash___Powdery_mildew',
  'Strawberry___healthy',
  'Strawberry___Leaf_scorch',
  'Tomato___Bacterial_spot',
  'Tomato___Early_blight',
  'Tomato___healthy',
  'Tomato___Late_blight',
  'Tomato___Leaf_Mold',
  'Tomato___Septoria_leaf_spot',
  'Tomato___Spider_mites Two-spotted_spider_mite',
  'Tomato___Target_Spot',
  'Tomato___Tomato_mosaic_virus',
  'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]

In [5]:
data_transforms = {
  'train': transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) 
  ]),
  'val': transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
  ]),
}

In [7]:
train_dir_original = os.path.join(base_dir, 'train')
val_dir_original = os.path.join(base_dir, 'val')
os.makedirs(train_dir_original, exist_ok=True)
os.makedirs(val_dir_original, exist_ok=True)

In [ ]:
if not os.listdir(train_dir_original) or not os.listdir(val_dir_original):
  # Copy data to the correct directories if not already done
  for category in classes:
    category_path = os.path.join(base_dir, 'original', category)
    images = os.listdir(category_path)

    # Ensure the category directories exist in train and val directories
    train_category_dir = os.path.join(train_dir_original, category)
    val_category_dir = os.path.join(val_dir_original, category)
    os.makedirs(train_category_dir, exist_ok=True)
    os.makedirs(val_category_dir, exist_ok=True)
    train_images = images[:int(len(images) * 0.8)]
    val_images = images[int(len(images) * 0.8):]
    for img in tqdm(train_images, desc=f"Copying train images for {category}"):
      src_path = os.path.join(category_path, img)
      dst_path = os.path.join(train_category_dir, img)
      if not os.path.exists(dst_path):
        shutil.copy(src_path, dst_path)
    for img in tqdm(val_images, desc=f"Copying val images for {category}"):
      src_path = os.path.join(category_path, img)
      dst_path = os.path.join(val_category_dir, img)
      if not os.path.exists(dst_path):
        shutil.copy(src_path, dst_path)

print("\nTrain Directory Structure:")
for root, dirs, files in os.walk(train_dir_original):
  print(root, "contains", len(files), "files")

print("\nValidation Directory Structure:")
for root, dirs, files in os.walk(val_dir_original):
  print(root, "contains", len(files), "files")

In [ ]:
train_dataset_original = datasets.ImageFolder(train_dir_original, transform=data_transforms['train'])
val_dataset_original = datasets.ImageFolder(val_dir_original, transform=data_transforms['val'])
train_loader_original = DataLoader(train_dataset_original, batch_size=32, shuffle=True)
val_loader_original = DataLoader(val_dataset_original, batch_size=32, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_original = EfficientNet.from_pretrained('efficientnet-b2')
num_classes_original = len(train_dataset_original.classes)
model_original._conv_stem = nn.Conv2d(1, model_original._conv_stem.out_channels, 
                             kernel_size=model_original._conv_stem.kernel_size, 
                             stride=model_original._conv_stem.stride, 
                             padding=model_original._conv_stem.padding, 
                             bias=False)
model_original._fc = nn.Linear(model_original._fc.in_features, num_classes_original)
model_original = model_original.to(device)
criterion_original = nn.CrossEntropyLoss()
optimizer_original = torch.optim.Adam(model_original.parameters(), lr=0.001)

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, device, num_epochs=50, patience=5):
  best_val_loss = float('inf')
  epochs_no_improve = 0
  model_save_path = '.pth'
  os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    #Training phase with progress bar
    train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", unit="batch")
    for inputs, labels in train_progress:
      inputs, labels = inputs.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item() * inputs.size(0)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      train_progress.set_postfix({"Loss": running_loss / total, "Acc": correct / total})
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
      for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_correct / val_total
    print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

    #Early stopping
    if val_loss < best_val_loss:
      best_val_loss = val_loss
      epochs_no_improve = 0
      torch.save(model.state_dict(), model_save_path)  # Save the best model
    else:
      epochs_no_improve += 1
    if epochs_no_improve >= patience:
      print("Early stopping triggered!")
      break

    return model

In [ ]:
def plot_confusion_matrix(cm, class_names):
  figure = plt.figure(figsize=(8, 8))
  sns.heatmap(cm, annot=True, cmap=plt.cm.Blues, fmt='g', xticklabels=class_names, yticklabels=class_names)
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.title('Confusion Matrix')
  return figure

In [ ]:
def evaluate_model(model, dataloader, device):
  model.eval()
  correct = 0
  total = 0
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for inputs, labels in dataloader:
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      all_preds.extend(predicted.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())
  accuracy = correct / total
  cm = confusion_matrix(all_labels, all_preds)
  cm_figure = plot_confusion_matrix(cm, class_names=dataloader.dataset.classes)

  # Print and show the confusion matrix
  plt.figure(figsize=(8, 8))
  sns.heatmap(cm, annot=True, cmap=plt.cm.Blues, fmt='g', xticklabels=dataloader.dataset.classes, yticklabels=dataloader.dataset.classes)
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.title('Confusion Matrix - Evaluation')
  plt.show()
  print(f'Accuracy: {accuracy:.4f}')

  return accuracy

In [ ]:
trained_model_original = train_model(model_original, criterion_original, optimizer_original, train_loader_original, val_loader_original, device, num_epochs=10, patience=3)

In [ ]:
evaluate_model(trained_model_original, val_loader_original, device)